# Декораторы

In [1]:
def foo(x):
    print(x + 1)


def decorator(foo):
    def _dec(*args, **kwargs):
        print("inside decorator")
        res = foo(*args, **kwargs)
        return res
    
    return _dec

In [3]:
foo(1)

2


In [4]:
foo = decorator(foo)
foo(1)

inside decorator
2


### В Python есть специальный синтаксис, который позволяет переместить модификацию функции ближе к моменту объявления функции:

In [5]:
@decorator
def foo(x):
    """
    foo description
    """
    print(x + 1)

In [6]:
foo(1)

inside decorator
2


### Есть небольшая проблема:

In [7]:
foo.__name__, foo.__doc__

('_dec', None)

#### Еще "пропадает" ```__module__```
#### Что делать?

In [8]:
def decorator(foo):
    def _dec(*args, **kwargs):
        res = foo(*args, **kwargs)
        return res
    
    _dec.__name__ = foo.__name__
    _dec.__doc__ = foo.__doc__
    _dec.__module__ = foo.__module__
    
    return _dec


@decorator
def foo(x):
    """
    foo description
    """
    print(x + 1)

In [9]:
foo.__name__, foo.__doc__

('foo', '\n    foo description\n    ')

#### Но это сложно, можно проще.
#### Воспользуемся декоратором ```wraps``` из пакета ```functools``` <i>(подробнее про этот пакет чуть позже)</i>

In [16]:
from functools import wraps


def decorator(foo):
    @wraps(foo)
    def _dec(*args, **kwargs):
        res = foo(*args, **kwargs)
        return res
    
    return _dec


@decorator
def foo(x):
    """
    foo description
    """
    print(x + 1)

In [17]:
foo.__name__, foo.__doc__

('foo', '\n    foo description\n    ')

### Декораторов может быть несколько:

In [18]:
def dec1(func):
    @wraps(func)
    def _dec(*args, **kwargs):
        print("inside dec1")
        res = func(*args, **kwargs)
        return res
        
    return _dec


def dec2(func):
    @wraps(func)
    def _dec(*args, **kwargs):
        print("inside dec2")
        res = func(*args, **kwargs)
        return res
        
    return _dec

In [19]:
@dec2
@dec1
def func(arg1, arg2):
    pass

In [20]:
func(1, 2)

inside dec2
inside dec1


#### Это же самое, что:

In [ ]:
def func(arg1, arg2):
    pass

In [ ]:
func = dec2(dec1(func))

In [ ]:
func(1, 2)

### Вопрос [?]

In [21]:
flag = False


def decorator(func):
    @wraps(func)
    def _dec(*args, **kwargs):
        print("inside _dec")
        res = func(*args, **kwargs)
        return res
    
    return func if flag else _dec


@decorator
def func():
    pass

In [22]:
func()

inside _dec


#### Что будет?

In [23]:
flag = True
func()

inside _dec


### В декоратор можно передавать аргументы:

In [32]:
def decorator_with_args(dec_argument):
    def _decorator(func):
        @wraps(func)
        def _dec(*args, **kwargs):
            print(f"inside decorator; {dec_argument}")
            res = func(*args, **kwargs)
            return res
        
        return _dec
    return _decorator

In [33]:
@decorator_with_args("hop hey lala ley")
def func(x):
    pass

In [34]:
func(None)

inside decorator; hop hey lala ley


#### Это то же самое, что:

In [ ]:
def decorator_with_args(dec_argument):
    def _decorator(func):
        @wraps(func)
        def _dec(*args, **kwargs):
            print(f"inside decorator; {dec_argument}")
            res = func(*args, **kwargs)
            return res
        
        return _dec
    return _decorator

In [ ]:
def func(x):
    pass

In [ ]:
decorator = decorator_with_args("hop hey lala ley")  # получим тут декоратор
func = decorator(func)  # получим модифицированную функцию

In [ ]:
func(None)

#### А если мы хотим опциональные аргументы?

In [29]:
def decorator_with_optional_arguments(func=None, *, dec_argument="default"):
    if func is None:
        return lambda func: decorator_with_optional_arguments(func, dec_argument=dec_argument)
    @wraps(func)
    def _dec(*args, **kwargs):
        print(f"inside decorator; {dec_argument}")
        res = func(*args, **kwargs)
        return res
    
    return _dec

In [30]:
@decorator_with_optional_arguments(dec_argument="Life is beatiful")
def func1():
    pass


func1()

inside decorator; Life is beatiful


In [31]:
@decorator_with_optional_arguments
def func2():
    pass


func2()

inside decorator; default


### Несколько примеров полезных декораторов

#### Посчитать, сколько раз выполнялась функция:

In [35]:
def profiled(func):
    @wraps(func)
    def inner(*args, **kwargs):
        inner.ncalls += 1
        return func(*args, **kwargs)
    inner.ncalls = 0
    return inner

In [36]:
@profiled
def f():
    pass


for i in range(1000):
    f()
    
    
print(f.ncalls)

1000


#### Декоратор, для вызова декорируемой функции только 1 раз

In [42]:
# TODO
def once(func):
    @wraps(func)
    def _dec(*args, **kwargs):
        _dec.cnt +=1
        if _dec.cnt == 2:
            raise Exception
        return func(*args, **kwargs)
    _dec.cnt = 0
    return _dec

In [43]:
@once
def f():
    pass
f()


* [contextlib.contextmanager](https://docs.python.org/3.5/library/contextlib.html#contextlib.contextmanager)
* [functools.lru_cache](https://docs.python.org/3.5/library/functools.html#functools.lru_cache)
* почти все в [pycontracts](https://andreacensi.github.io/contracts/)
* @classmethod и @staticmethod
* @property
* @abstractmethod

...

In [44]:
f()

NameError: name 'Error' is not defined

# Области видимости

In [24]:
print(min)  # build-in

a = 0  # global
b = 1  # global


def g():
    a = -1  # enclosing
    b = -2  # enclosing
    
    def f():
        a = 1  # local
        b = 2  # local

        print("locals: ", locals())

        print("a in globals=", globals()["a"], sep="")
        print("b in globals=", globals()["b"], sep="")
        
    f()
    
    
g()

<built-in function min>
locals:  {'a': 1, 'b': 2}
a in globals=0
b in globals=1


### А что с присваиванием?

In [25]:
a = -1  # global


def f():
    try:
        a += 1  # local
    except UnboundLocalError:
        print("Catch UnboundLocalError")


f()

Catch UnboundLocalError


#### Можно ли как-то присвоить? Да!

In [26]:
a = -1  # global


def f():
    globals()["a"] += 1  # local


f()
print(a)

0


#### Или так:

In [27]:
a = -1  # global


def f():
    global a
    a += 1  # local
    print(a)


f()

0


#### Для ```enclosing```, соответственно, вот так вот:

In [28]:
def g():
    a = -1  # enclosing

    def f():
        nonlocal a
        a += 1  # local

    f()
    print(a)
    
    
g()

0


# Дескрипторы

### @property

In [56]:
class MoneySaver:
    def __init__(self, exchange_rate):
        self.exchange_rate = exchange_rate
        self._copilka = 0
        
    @property
    def copilka(self):
        return self._copilka * self.exchange_rate
    
    @copilka.setter
    def copilka(self, x):
        self._copilka += x / self.exchange_rate

In [57]:
saver = MoneySaver(100)
saver.copilka = 10
print(saver.copilka)

10.0


In [45]:
class SomeBank:
    def __init__(self, start):
        self._rubles = start
        
    @property
    def rubles(self):
        return self._rubles
    
    @rubles.setter
    def rubles(self, value):
        if value > 0:
            self._rubles = value
        else:
            raise Exception('You shall not pass!')

    @rubles.deleter
    def rubles(self, value):
        del self._rubles

In [46]:
interv = SomeBank(6000)

In [47]:
interv.rubles

6000

In [48]:
interv.rubles = 4242

In [49]:
interv.rubles = -32

Exception: You shall not pass!

### \_\_get__(), \_\_set__() и \_\_delete__(), 

In [93]:
class Descr:
    def __get__(self, instance, owner):
        print(instance, owner)
        
    def __set__(self, instance, value):
        print(instance, value)
        
    def __delete__(self, instance):
        print(instance)
        
    
class A:
    attr = Descr()

In [94]:
A().attr

<__main__.A object at 0x7f30e273a588> <class '__main__.A'>


In [95]:
A.attr

None <class '__main__.A'>


In [96]:
A().attr = 42

<__main__.A object at 0x7f30e2718400> 42


#### Что будет?

In [102]:
A.attr = 42
A().__dict__


{}

In [90]:
del A().attr

AttributeError: attr

In [91]:
del A.attr

### Хм

* `instance` -- экз. класса
* `attr` -- атрибут (который дескриптор)
* `descr = cls.__dict__["attr"]` -- сам дескриптор

In [97]:
instance = A()
descr = A.__dict__["attr"]

####  Тогда:

In [98]:
A.attr
descr.__get__(None, A)

None <class '__main__.A'>
None <class '__main__.A'>


In [99]:
instance.attr
descr.__get__(instance, A)

<__main__.A object at 0x7f30e2718940> <class '__main__.A'>
<__main__.A object at 0x7f30e2718940> <class '__main__.A'>


In [100]:
instance.attr = 42
descr.__set__(instance, 42)

<__main__.A object at 0x7f30e2718940> 42
<__main__.A object at 0x7f30e2718940> 42


In [101]:
del instance.attr
descr.__delete__(instance)

### Вопрос [?]

#### Где хранить данные дескриптору?

### Еще Вопрос [?]

#### Как вы думаете, как работают методы в Python?

`types.MethodType`

* http://stupidpythonideas.blogspot.com/2013/06/how-methods-work.html
* http://igorsobreira.com/2011/02/06/adding-methods-dynamically-in-python.html